In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from matplotlib import rcParams
import pandas as pd

# --- Estilo global coherente ---
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Open Sans', 'Segoe UI', 'DejaVu Sans', 'Arial']
rcParams['font.size'] = 11

# --- Parámetros del problema ---
r, c, v = 8, 5, 4
lambda_param = 1 / 10

# --- Funciones ---
def beneficio(x, xi):
    return r * min(x, xi) + v * max(0, x - xi) - c * x

def saa_objective(x, xi_samples):
    return -np.mean([beneficio(x, xi) for xi in xi_samples])

def expected_cost(x):
    expected_min = (1 - np.exp(-lambda_param * x)) / lambda_param
    expected_max0 = x - expected_min
    return - (r * expected_min + v * expected_max0 - c * x)

def saa_cost(x, xi_samples):
    return np.mean([- (r * min(x, xi) + v * max(0, x - xi) - c * x) for xi in xi_samples])

# --- Rango de cantidades ---
x_vals = np.linspace(0, 40, 50)
true_costs = np.array([expected_cost(x) for x in x_vals])

# --- Tamaños de muestra ---
N_grid = [10, 100, 1000, 10000, 100000]
saa_curves = {}
np.random.seed(0)

for N in N_grid:
    xi_samples = np.random.exponential(scale=1 / lambda_param, size=N)
    saa_curves[N] = np.array([saa_cost(x, xi_samples) for x in x_vals])

# --- Convergencia de la solución óptima ---
N_values = [10, 100, 1000, 10000, 100000, 1000000]
x_opt_values = []
for N in N_values:
    xi_samples = np.random.exponential(scale=1 / lambda_param, size=N)
    res = minimize_scalar(lambda x: saa_objective(x, xi_samples),
                          bounds=(0, 1.5 * np.max(xi_samples)),
                          method='bounded')
    x_opt_values.append(res.x)

# --- Solución teórica ---
cuantil = (r - c) / (r - v)
x_star = - (1 / lambda_param) * np.log(1 - cuantil)

# --- Subplot conjunto: curvas f_N y convergencia x_N ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), facecolor='white')

# (1) Curvas f_N(x)
ax1.plot(x_vals, true_costs, color="#1e3a5f", label="Coste esperado teórico", linewidth=2.5)
for N in N_grid:
    ax1.plot(x_vals, saa_curves[N], linestyle='--', linewidth=1.5, alpha=0.85,
             label=f"SAA ($N={N}$)")
ax1.set_xlabel("Cantidad preparada $x$")
ax1.set_ylabel("Coste esperado")
#ax1.set_title("Convergencia puntual de $f_N(x)$ a $\\mathbb{E}[f(x)]$")
ax1.legend()
ax1.grid(True, alpha=0.2)

# (2) Convergencia de x_N
ax2.plot(N_values, x_opt_values, 'o-', color="#1e3a5f", linewidth=2.5,
         markersize=6, markerfacecolor="#1e3a5f", label=r"$x_N^*$")
ax2.axhline(y=x_star, color="#8C1818", linestyle='--', linewidth=2,
            alpha=0.85, label=fr"$x^* \approx {x_star:.2f}$ (teórico)")
ax2.set_xscale('log')
ax2.set_xticks(N_values)
ax2.set_xticklabels([r"$10^1$", r"$10^2$", r"$10^3$", r"$10^4$", r"$10^5$", r"$10^6$"])
ax2.set_xlabel("Tamaño de muestra $N$")
ax2.set_ylabel("Solución óptima $x$")
#ax2.set_title("Convergencia de $x*_N$ a $x^*$")
ax2.grid(True, alpha=0.2)
ax2.legend()

plt.tight_layout()

# Crear DataFrame con X y cada curva
df = pd.DataFrame({"x": x_vals, "coste_teorico": true_costs})
for N in N_grid:
    df[f"SAA_N={N}"] = saa_curves[N]

# Copiar al portapapeles (listo para pegar en Excel)
df.to_clipboard(index=False)
# --- Datos para la gráfica de convergencia (derecha) ---
df_conv = pd.DataFrame({"N": N_values, "x_opt": x_opt_values})
df_conv = df_conv.round(2)
df_conv.to_clipboard(index=False)  # listo para pegar en Excel / texto
plt.show()



In [ ]:
# --- Experimento de la maldición del optimizador ---
N_vals = [10, 30, 100, 300, 1000]
M = 500
optima_saa = {}

for N in N_vals:
    values = []
    for _ in range(M):
        xi_samples = np.random.exponential(scale=1/lambda_param, size=N)
        costs = [saa_cost(x, xi_samples) for x in x_vals]
        values.append(min(costs))
    optima_saa[N] = values

# --- Valor óptimo verdadero ---
true_opt = min(true_costs)

# --- Gráfico de la maldición del optimizador ---
means = [np.mean(optima_saa[N]) for N in N_vals]
stds = [np.std(optima_saa[N]) for N in N_vals]

plt.figure(figsize=(8, 5))
plt.errorbar(N_vals, means, yerr=stds, fmt='o-', color='#1e3a5f', capsize=4,
             label="Media de $\\nu_N^*$ (SAA)", linewidth=2)
plt.axhline(y=true_opt, color='#8C1818', linestyle='--', linewidth=2,
            label=fr"$\nu^* \approx {true_opt:.2f}$")
plt.xscale('log')
plt.xlabel("Tamaño de muestra $N$")
plt.ylabel("Valor óptimo estimado")
plt.title("Sesgo del valor óptimo en SAA (maldición del optimizador)")
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.3)

plt.tight_layout()
plt.show()